# HuggingFace Transformers
## 01 Basic toturial
제작 : 이유경 (yukyung_lee@korea.ac.kr)

In [1]:
! pip install transformers==4.5.0 -q

     |████████████████████████████████| 2.1 MB 9.5 MB/s 
     |████████████████████████████████| 3.3 MB 65.7 MB/s 
     |████████████████████████████████| 895 kB 57.3 MB/s 


## 💻 Basic tutorial
* 아래와 같은 내용을 정리합니다
    * 01 : Tokenizer 불러오기
    * 02 : Config 불러오기
    * 03 : Pretrained model 불러오기
    * 04 : Huggingface Trainer로 학습 진행하기

### 01 Tokenizer 불러오기

텍스트 데이터를 모델이 알아들을 수 있는 형태로 변환하기 위해서는 `tokenization` 과정을 거쳐 encoding을 진행해야합니다. 

이 때 text data를 token화 하고 특정 숫자로 encoding 하는 과정을 모두 수행하는것이 transformers tokenizer 역할입니다.

> Q : PLM(Pretrained language model)dl 없었던 시기에는 tokenizer를 어떻게 사용했을까요 ? 

> A : 내가 가지고 있는 데이터를 기반으로 parsing을 진행(성능 좋은 Parser를 사용했음)한 후 dictionary를 데이터별로 만들어 직접 숫자를 부여했습니다



In [1]:
from transformers import AutoTokenizer

example = "나는 학생이다."

# tokenization 결과를 보여줍니다.
model_name = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading: 100%|██████████| 570/570 [00:00<00:00, 147kB/s]
Downloading: 100%|██████████| 213k/213k [00:00<00:00, 261kB/s] 
Downloading: 100%|██████████| 436k/436k [00:01<00:00, 385kB/s] 
Downloading: 100%|██████████| 29.0/29.0 [00:00<00:00, 16.4kB/s]


In [2]:
# tokenization 결과를 보여줍니다.
# bert-base-cased는 영어에 대한 tokenizer이므로 한글을 전혀 이해하지 못함
# 따라서 tokenizer output은 [unk]으로 나옴
print('tokenization 결과 : ', tokenizer.tokenize(example))
print('tokenization + encoding 결과 : ', tokenizer.encode(example))

tokenization 결과 :  ['[UNK]', '[UNK]', '.']
tokenization + encoding 결과 :  [101, 100, 100, 119, 102]


In [3]:
example = "나는 고려대학교 학생이다."

model_name = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading: 100%|██████████| 425/425 [00:00<00:00, 176kB/s]
Downloading: 100%|██████████| 248k/248k [00:00<00:00, 303kB/s] 
Downloading: 100%|██████████| 495k/495k [00:01<00:00, 493kB/s]
Downloading: 100%|██████████| 289/289 [00:00<00:00, 139kB/s]


In [4]:
# tokenization 결과를 보여줍니다.
# klue/bert-base는 한글에 대한 tokenizer이므로 한글을 인식할 수 있음
# 언어에 알맞는 적절한 tokenizer를 사용해야함
print('tokenization 결과 : ', tokenizer.tokenize(example))
print('tokenization + encoding 결과 : ', tokenizer.encode(example))

tokenization 결과 :  ['나', '##는', '고려대', '##학교', '학생', '##이다', '.']
tokenization + encoding 결과 :  [2, 717, 2259, 8943, 4622, 3767, 28674, 18, 3]


**⛔️ tokenizer 사용시 주의사항**

1. train data의 언어를 이해 할 수 있는 tokenizer인지 확인
2. 사용하고자 하는 pretrained model과 동일한 tokenizer인지 확인
  
  > 적절한 tokenizer를 사용하지 않을 경우 **vocab size mismatch**에러가 발생하거나 **special token이 `[unk]`**으로 처리되는 🤦🏻‍♀️대참사🤦🏻‍♂️가 벌어질 수 있음
3. 단어의 개수와 special token이 완전히 일치하는 모델은 (예를들어 klue의 roberta, bert) tokenizer를 cross로 사용 **'할 수도'** 있지만 옳은 방법은 아님
  * 첨언하자면, 공개된 영어 bert와 roberta는 tokenizer가 호환되지 않습니다. (bert vocab 28996개, roberta vocab 50265개)
  * klue bert는 동일한 기관에서 생성된 모델이므로 32000개로 총 vocab 사이즈가 동일하지만 이는 우연의 일치입니다.



------

### 02 Config 불러오기

**사전 학습 모델을 사용하기 위해서는 사전학습 모델이 가진 setting을 그대로 가져와야합니다.**

모델마다 vocab size, hidden dimension등 각각의 파라미터 세팅이 상이하므로 

transformers는 이 정보를 Config로 쉽게 불러올 수 있는 기능을 제공합니다.

모델명+Config.from_pretrained가 가장 기본적인 형태였지만, 요즘은 `Auto` class로 더욱 편리하게 configuration을 가져올 수 있습니다.




In [6]:
from transformers import AutoConfig

model_name =  'klue/bert-base'

# pretrained 모델과 동일한 configuration을 가져옵니다.
model_config = AutoConfig.from_pretrained(model_name)

In [7]:
model_config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.4.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

**⛔️ config 사용시 주의사항**

어떤 경우에는 config를 수정하여 사용하기도 하는데, 바꾸어도 되는 config와 바꾸지 말아야 하는 config가 정해져 있습니다.

**바꾸면 안되는 config**
* Pretrained model 사용시 hidden dim등 이미 정해져 있는 모델의 아키텍쳐 세팅은 수정하면 안됩니다.
* 이를 수정해버릴 경우 에러가 발생하거나, 잘못된 방향으로 학습 될 수 있습니다.

**바꾸어도 되는 config**
* vocab의 경우 special token을 추가한다면 config를 추가한 vocab의 개수만큼 추가하여 학습해야합니다.
* downstream task를 위해 몇가지 config를 추가할 수도 있습니다. (아래에서 예시를 살펴봅시다)






**sequence classification 모델을 config 세팅의 예시로 설명합니다.**

transformers document를 보면 사용하는 모델별로 미리 정의되어야 하는 config들에 대해 알려주고 있습니다

```
class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config

        self.bert = BertModel(config)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
```
classification을 수행하기 위해서는 몇개의 class로 분류해야하는지 정보를 제공해야합니다. 따라서  num_labels라는 정보를 꼭 기입해주어야 하는 상황입니다. 

그럼 아래의 예시를 살펴볼까요 ?


In [8]:
'''
현재 상황은 아래와 같습니다.
1) 사용 모델 : Sequence classification 모델을
2) special token 2개 추가함
3) label은 총 10개

case 1. 원하는 config의 값을 수정하는 케이스 (2)
case 2. downstream task를 위해 추가해야하는 config 케이스 (3)
'''

model_name =  'klue/bert-base'

# case 1
# config를 추가하는 방법은 두가지로 가능함 

# [1] 호출 후 직접 config의 값을 수정하는 방법
model_config = AutoConfig.from_pretrained(model_name)
model_config.vocab_size = model_config.vocab_size + 2
print('case 1 - [1] : ', model_config)

# [2] 호출과 동시에 수정하는 방법 
# 하지만, vocab 수정은 해당 코드로 진행하는것을 권장하지 않음
# Advanced tutorial의 token 추가하기에서 자세히 다룸
model_config = AutoConfig.from_pretrained(model_name , vocab_size=32002)

print('case 1 - [2] : ', model_config)

case 1 - [1] :  BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.4.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32002
}

case 1 - [2] :  BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_h

In [9]:
# case 2 (sequence classification을 위해 num_labels를 설정하기)
# config를 추가하는 방법은 두가지로 가능함 

# [1] 호출 후 직접 config의 값을 수정하는 방법
model_config = AutoConfig.from_pretrained(model_name)
model_config.num_labels = 10

# print('case 1 - [1] : ', model_config)

# [2] 호출과 동시에 수정하는 방법
model_config = AutoConfig.from_pretrained(model_name , num_labels = 10)

# print('case 1 - [2] : ', model_config)

📣 **아무런 값이나 config에 추가할 수 있을까요 ?**

임의로 만들어진 config key는 config에 추가 될수는 있지만 모델 학습에 사용되지 않습니다.

In [10]:
# 해당 방법으로 생성하면 config에 추가 됨
model_config.anything = 'yeah!!'

model_config

BertConfig {
  "anything": "yeah!!",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "LABEL_9": 9
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.4.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "v

In [11]:
# 해당 방법으로 생성하면 config에 추가되지 않음 (모델 학습에 사용되지 않는 key 이므로)
# AutoConfig.from_pretrained(model_name/path , **kwargs)
# model name 이후에 keyward argument(**kwargs)로 인자를 받을 수 있으니 딕셔너리 형태로 값을 받을 수도 있습니다. 
model_config = AutoConfig.from_pretrained(model_name , anything = 'yeah!!')
model_config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.4.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

In [12]:
# 참고 : 원하는 custom_config로 config 전체를 업데이트 해주는 방법도 있음

custom_config = {
    #원하는 값을 넣으면 됩니다
    "vocab_size": 32000
    }

model_config = AutoConfig.from_pretrained(model_name)
model_config.update(custom_config)

---

### 03 Pretrain model 불러오기

**transformers의 가장 강력한 기능은 사전학습된 모델을 쉽게 불러오고, 사용할 수 있다는 것입니다.**

단 세 줄로 원하는 pretrained model을 불러오고 사용할 수 있습니다.

(물론 원하는 모델이 Huggingface transformers에 공개되어 있어야하겠죠 !😎)

우리는 해당모델을 그대로 사용할 수도 있고, 추가적으로 학습을 진행하여 내 데이터에 맞는 모델로 사용할수도 있습니다.

* `.from_config()` 는 config 그대로 모델을 가져오는 method 입니다. 즉 사전학습된 weight을 가져오는게 아니니 주의해야합니다.
* `.from_pretrained()` 는 model config에 해당하는 모델을 가져오고, **사전학습된 weight를 가져옵니다**. 스스로 학습한 모델을 불러오려면 model_name 부분에 model이 저장된 directory를 입력하면 됩니다.



**transformers는 두가지 타입의 모델을 제공하고 있습니다.**

* 기본 모델
  
  : hidden state가 출력되는 기본 모델

* downstream task 모델
  
  : 일반적인 task를 쉽게 수행할 수 있도록 미리 기본 모델 + head가 설정된 모델
  
  : output은 task에 적합한 dimension으로 미리 정의되어있음

**+ ) transformers 사용자들이 공개한 checkpoint를 마음껏 사용할 수 있습니다.**

  https://huggingface.co/models

  models에서 이미 배포된 모델을 손쉽게 사용할 수 있으니 목적에 맞는 모델을 찾아 사용해보는것도 좋습니다.

In [13]:
from transformers import AutoConfig, AutoModelForQuestionAnswering
# Download configuration from huggingface.co and cache.

model_name =  'klue/bert-base'

# pretrained 모델과 동일한 configuration을 가져옵니다.
model_config = AutoConfig.from_pretrained(model_name)

# 모델을 정의합니다.

# option 1 : config에서 정의한 모델을 가져오기 (initial)
# model = AutoModelForQuestionAnswering.from_config(config)

# option 2 : config에서 정의한 사전학습된 모델을 가져오기 (pretrained)
model = AutoModelForQuestionAnswering.from_pretrained(
        model_name, config=model_config
    )


Downloading: 100%|██████████| 445M/445M [00:24<00:00, 18.2MB/s]
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of B

**이제 우리는 가장 기본적인 tokenizer, config, model를 loading하는 방법에 대해 배웠습니다.**

원리를 알게 되었으니 아래의 방법으로 세가지를 한번에 load하면 됩니다.

In [ ]:
model_name = 'klue/bert-base'

config = AutoConfig.from_pretrained(
    model_name,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
)
model = AutoModelForQuestionAnswering.from_pretrained(
    model_name,
    config=config,
)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model chec

----

## 04 Huggingface Trainer로 학습 진행하기
> Trainer is all you need ? 🤔

  반복되는 Training loop를 효과적으로 모듈화 시켜놓은것이 transformers의 trainer입니다. 
  
  덕분에 우리는 매 모델을 학습하기 위해 training loop를 구현하는 과정을 단 몇줄만에 해결할 수 있습니다.


아래와 같은 structure로 trainer를 사용할 수 있습니다.

* TrainingArguments 설정
* Trainer 호출
* 학습 / 추론 진행

```
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model_name = 'klue/bert-base'

model = AutoModelForQuestionAnswering.from_pretrained(model_name)

args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)


trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()
```



 **Trainer가 '항상' 좋을까요 ?**

* **다음과 같은 모듈의 코드는 legacy가 존재할 수 밖에 없습니다.** 

  : 따라서 버전이 바뀔 때 마다 변동되는 사항이 많아지고 코드를 지속적으로 수정해야하는 단점이 존재합니다. 

  : pytorch lightning이 대표적으로 이러한 문제를 겪고 있으며, transformers도 예외는 아닙니다.

  : 따라서 Trainer는 모든 상황에서 정답이 될 수는 없습니다

* **최대한 편리함을 이용하되, 동작 원리를 살펴보는 과정이 매우 중요합니다.**

* Trainer의 구조를 살펴보고, 내가 학습할 모델을 위한 Trainer를 만들어보는것도 좋은 방법입니다 
  * Trainer에 원하는 함수 오버라이딩 하여 수정하기 (general task에 적합)
  * Custome Trainer 만들어보기 (general task가 아닌경우 유용함)